In [1]:
import math
import numpy as np
import shapefile
import time

In [2]:
test = [1,2,3,4]
print(test[:-1])

[1, 2, 3]


In [3]:
class LinkNode:
    def __init__(self, idx, position, link_road):
        self.idx = idx
        self.position = position
        self.link_road = link_road
    def add_road(self, road_id):
        self.link_road.append(road_id)
    def __eq__(self,other):
        return self.idx == other.idx

class Road:
    def __init__(self, idx, start_node, end_node, length):
        self.idx = idx
        self.start_node = start_node
        self.end_node = end_node
        self.length = length
    def assign_jam(self, jam_level):
        self.jam_level = jam_level
    def __eq__(self,other):
        return self.idx == other.idx

In [4]:
def cal_tpsg(station, target):
    cnt = 0
    for psg in station.psg_list:
        if psg.a_station == target.station_id:
            cnt += 1
    return cnt



def distance_score(start, target, k_d):
    distance = math.sqrt((start.position[0]-target.position[0])**2+(start.position[1]-target.position[1])**2)
    return k_d*distance

def Passenger_score(start, target, k_p):
    return -k_p*(math.exp(target.psg_count)-1)

def distrib_score(bus, start, target, k_a):
    # 在计算分数前，先将当前站点加入队列
    t_psg, all_psg = 0,0
    for station in bus.plan_route:
        t_psg += cal_tpsg(station, target)
        all_psg += station.psg_count
    a_s = t_psg/all_psg
    return k_a * math.log(a_s)

def waittime_score(target, k_w):
    average_wait_time = sum([(time.time()-psg.b_time) for psg in target.psg_list])/target.psg_count

    return - k_w *[np.exp(average_wait_time)-1]

def calculate_score(start, target, k):
    return distance_score(start, target, k[0])+Passenger_score(start, target, k[1])+distrib_score(start, target, k[2])+ waittime_score(target, k[3])
    


In [5]:
class Station:
    def __init__(self, station_id, psg_list, position):
        self.station_id = station_id
        self.psg_list = psg_list
        self.position = position
        self.psg_count = len(self.psg_list)
    def add_psg(self, psg):
        self.psg_list.append(psg)
    def __eq__(self,other):
        return self.station_id == other.station_id

class Passengers:
    def __init__(self, idx,state, b_station, a_station):
        self.idx = idx
        self.state = state 
        self.b_station = b_station
        self.a_station = a_station 
        self.b_time = time.time()
        self.board_bus = None
    def change_state(self, state):
        self.state = state 
    def __eq__(self,other):
        return self.idx == other.idx


class Bus:
    def __init__(self, ID, state, N, psg_list, plan, c_station):
        self.ID = ID 
        self.state = 0
        self.N = 20
        self.psg_list = []
        self.plan_route = []
        self.c_station = station
    def __eq__(self,other):
        return self.ID == other.ID

    def choose_next_station(self, station_set:list,station_dict:dict, k_set:list):
        station_score = []
        for s in self.plan_route:
            station_set.remove(s)
        for station in station_set:
            station_score.append(calculate_score(station_dict[self.c_station],station_dict[station], k_set))
        min_value = min(station_score) # 求列表最小值
        min_idx = station_score.index(min_value) # 求最小值对应索引
        
        self.plan_route.append(station_set[min_idx])
        return station_set[min_idx]

    def state_update(self, station_dict:dict):
        if len(plan_route) == 0 or len(plan_route) == 1:
            return station_dict
        else:
            c_station = self.plan_route[-2]
            n_station = self.plan_route[-1]
            for psg in self.psg_list:
                if psg.a_station == c_station:
                    self.psg_list.remove(psg)
            max_board = self.N - 2*len(self.plan_route)
            cnt_board = 0
            for station in self.plan_route:
                for psg in station_dict[station].psg_list:
                    if psg.a_station == station:
                        station_dict[station].psg_list.remove(psg)
                        self.psg_list.append(psg)
                        cnt_board +=1
                        if cnt_board >= max_board or len(self>psg_list) > self.N:
                            break
                if cnt_board >= max_board or len(self>psg_list) > self.N:
    
                    break
        return station_dict            
        


In [6]:
def capacity_manage(avail_bus:list, station_set:list, station_dict:dict, k_set:list, max_route:int):
    moving_bus = []
    while len(avail_bus) != 0:
        for bus in avail_bus:
            next_station = bus.choose_next_station(station_set, station_dict, k_set)
            station_dict = self.state_update(station_dict)
            if len(bus.plan_route) > max_route:
                avail_bus.remove(bus)
                moving_bus.append(bus)
    return moving_bus
             

In [7]:
def board_simulate(moving_bus:list, station_dict:dict):
    for bus in moving_bus:
        bus.psg_list = []
        for idx in range(len(bus.plan_route)):
            station = bus.plan_route[idx]
            for psg in station_dict[station].psg_list:
                if len(bus_psg_list) >= bus.N:
                    break
                if psg.state == 0 and psg.a_station in plan_route[idx+1:]:
                    psg.state = 1
                    psg.board_bus = bus.ID
            if len(bus_psg_list) >= bus.N:
                break
    return station_dict
